In [20]:
#*************************
import pickle
import pandas as pd
import numpy as np
import sqlalchemy
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
import json
import requests
import time
from IPython.display import display
import sys

In [21]:
#*************************
url = "dbbikes.cvvkn6jkfbdf.eu-west-1.rds.amazonaws.com"
port = "3306"
db = "dbbikes"
user = "SEGroup25"
password = "SEGroup25?"



In [22]:
#*************************
engine = create_engine(f"mysql+mysqldb://{user}:{password}@{url}:{port}/{db}", echo=True)


In [23]:
#*************************
availability_sql = pd.read_sql("select * from dbbikes.availability", engine)

2023-04-14 18:09:12,989 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-04-14 18:09:12,991 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-14 18:09:13,022 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-04-14 18:09:13,023 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-14 18:09:13,040 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-04-14 18:09:13,043 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-14 18:09:13,088 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2023-04-14 18:09:13,091 INFO sqlalchemy.engine.Engine [generated in 0.00340s] ('dbbikes', 'select * from dbbikes.availability')
2023-04-14 18:09:13,127 INFO sqlalchemy.engine.Engine select * from dbbikes.availability
2023-04-14 18:09:13,128 INFO sqlalchemy.engine.Engine [raw sql] ()


In [24]:
#*************************
weather_sql = pd.read_sql("select * from dbbikes.weather", engine)

2023-04-14 18:09:15,197 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2023-04-14 18:09:15,198 INFO sqlalchemy.engine.Engine [cached since 2.11s ago] ('dbbikes', 'select * from dbbikes.weather')
2023-04-14 18:09:15,236 INFO sqlalchemy.engine.Engine select * from dbbikes.weather
2023-04-14 18:09:15,236 INFO sqlalchemy.engine.Engine [raw sql] ()


In [25]:
#*************************
weather_sql.shape[0]

1239

In [26]:
#*************************
availability_sql.shape

(518927, 4)

In [27]:
#*************************
availability_sql.head(5)

,available_bike_stands,available_bikes,last_update,number
0,23,7,2147483647,42
1,14,6,2147483647,30
2,21,12,2147483647,54
3,21,14,2147483647,108
4,14,16,2147483647,20


In [28]:
#*************************
# group by "category" and select first 300 rows of each group
df_trimmed = availability_sql.groupby('number').tail(weather_sql.shape[0])

# reset index of trimmed dataframe
df_trimmed = df_trimmed.reset_index(drop=True)

In [29]:
#*************************
df_trimmed.shape # output: 3

(141399, 4)

In [30]:
#*************************
df_trimmed['index'] = df_trimmed.groupby('number').cumcount()
merged_df = df_trimmed.join(weather_sql,on='index', how='left')
merged_df.head()

,available_bike_stands,available_bikes,last_update,number,index,time,cord_long,cord_lat,weather_main,main_temp,main_humidity,wind_speed,wind_degree,sys_sunset,sys_sunrise
0,0,1,2147483647,507,0,2023-03-30 13:53:20,-6.2672,53.344,Clouds,14.61,71.0,4.12,220.0,2023-03-30 18:54:48,2023-03-30 06:04:13
1,0,1,2147483647,507,1,2023-03-30 13:55:03,-6.2672,53.344,Clouds,14.61,71.0,4.12,220.0,2023-03-30 18:54:48,2023-03-30 06:04:13
2,0,1,2147483647,507,2,2023-03-30 14:11:04,-6.2672,53.344,Clouds,14.95,72.0,4.92,149.0,2023-03-30 18:54:48,2023-03-30 06:04:13
3,0,1,2147483647,507,3,2023-03-30 14:11:26,-6.2672,53.344,Clouds,14.95,72.0,4.92,149.0,2023-03-30 18:54:48,2023-03-30 06:04:13
4,0,1,2147483647,507,4,2023-03-30 14:15:02,-6.2672,53.344,Clouds,14.95,72.0,4.92,149.0,2023-03-30 18:54:48,2023-03-30 06:04:13


In [31]:
#*************************
label_encoder = preprocessing.LabelEncoder()
merged_df['weather_main'] = label_encoder.fit_transform(merged_df['weather_main'])

In [32]:
#*************************
merged_df.tail()

,available_bike_stands,available_bikes,last_update,number,index,time,cord_long,cord_lat,weather_main,main_temp,main_humidity,wind_speed,wind_degree,sys_sunset,sys_sunrise
141394,11,9,2147483647,39,1238,2023-04-14 17:05:02,-6.2672,53.344,1,12.48,56.0,5.81,274.0,2023-04-14 19:21:53,2023-04-14 05:28:33
141395,22,18,2147483647,83,1238,2023-04-14 17:05:02,-6.2672,53.344,1,12.48,56.0,5.81,274.0,2023-04-14 19:21:53,2023-04-14 05:28:33
141396,8,32,2147483647,92,1238,2023-04-14 17:05:02,-6.2672,53.344,1,12.48,56.0,5.81,274.0,2023-04-14 19:21:53,2023-04-14 05:28:33
141397,19,11,2147483647,21,1238,2023-04-14 17:05:02,-6.2672,53.344,1,12.48,56.0,5.81,274.0,2023-04-14 19:21:53,2023-04-14 05:28:33
141398,13,17,2147483647,88,1238,2023-04-14 17:05:02,-6.2672,53.344,1,12.48,56.0,5.81,274.0,2023-04-14 19:21:53,2023-04-14 05:28:33


In [33]:
#*************************
merged_df['month'] = merged_df['time'].dt.month
merged_df['dayofweek'] = merged_df['time'].dt.dayofweek
merged_df['hour'] = merged_df['time'].dt.hour
merged_df['minute'] = merged_df['time'].dt.minute

In [34]:
print(merged_df.isnull().sum().sum())

0


In [35]:
#*************************
merged_df.dtypes

available_bike_stands             int64
available_bikes                   int64
last_update                       int64
number                            int64
index                             int64
time                     datetime64[ns]
cord_long                       float64
cord_lat                        float64
weather_main                      int64
main_temp                       float64
main_humidity                   float64
wind_speed                      float64
wind_degree                     float64
sys_sunset               datetime64[ns]
sys_sunrise              datetime64[ns]
month                             int64
dayofweek                         int64
hour                              int64
minute                            int64
dtype: object

In [36]:
merged_df.shape

(141399, 19)

In [45]:
#*************************
X = merged_df[['number','month','hour','minute','weather_main','main_temp','main_humidity','wind_speed','dayofweek']]
Y = merged_df.available_bikes
X_train, X_test,y_train,y_test = train_test_split(X, Y, test_size=0.3,random_state=0)
bikes_model = RandomForestRegressor(n_estimators=merged_df['number'].nunique())

bikes_model.fit(X_train, y_train)
y_pred = bikes_model.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})


print('Test Set Predictions for Station: ')
print(df)
print()

#Accuracy score
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(y_test, y_pred))

# Write to a pickle file
with open('bikes_model.pkl', 'wb') as handle:
    pickle.dump(bikes_model, handle, pickle.HIGHEST_PROTOCOL)

Test Set Predictions for Station: 
        Actual  Predicted
93968        7   7.565217
37774        0   0.982609
128265      17  16.713043
15836        1   3.052174
118060      15  14.939130
...        ...        ...
97316        7   6.026087
127770       2   2.182609
87305       26  26.000000
88207       20  17.713043
89067        4   4.060870

[42420 rows x 2 columns]

Mean Absolute Error: 1.2269390976364716
Mean Squared Error: 6.149181771269215
Root Mean Squared Error: 2.4797543772053747
The accuracy score :  0.924460075562352


In [44]:
A = merged_df[['number','month','hour','minute','weather_main','main_temp','main_humidity','wind_speed','dayofweek']]
B = merged_df.available_bike_stands
A_train, A_test,b_train,b_test = train_test_split(A, B, test_size=0.3,random_state=0)
stands_model = RandomForestRegressor(n_estimators=merged_df['number'].nunique())

stands_model.fit(X_train, b_train)
b_pred =stands_model.predict(A_test)
df = pd.DataFrame({'Actual': b_test, 'Predicted': b_pred})

print('Test Set Predictions for Station: ')
print(df)
print()

#Accuracy score
print('Mean Absolute Error:', metrics.mean_absolute_error(b_test, b_pred))
print('Mean Squared Error:', metrics.mean_squared_error(b_test, b_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(b_test, b_pred)))
print ("The accuracy score : ",metrics.r2_score(b_test, b_pred))

with open('stands_model.pkl', 'wb') as handle:
    pickle.dump(stands_model, handle, pickle.HIGHEST_PROTOCOL)

Test Set Predictions for Station: 
        Actual  Predicted
93968       33  32.130435
37774       20  19.078261
128265      12  11.278261
15836       39  35.600000
118060      25  25.130435
...        ...        ...
97316       32  31.060870
127770      38  38.043478
87305        4   4.000000
88207       10  11.191304
89067       26  25.973913

[42420 rows x 2 columns]

Mean Absolute Error: 1.2493417788983865
Mean Squared Error: 6.492727352454392
Root Mean Squared Error: 2.5480830740881255
The accuracy score :  0.9386844851836653


In [ ]:
""""
@app.route("/occupancy/<int:station_id>")
def get_occupancy(station_id):
    engine = create_engine(f"mysql+mysqldb://{user}:{password}@{url}:{port}/{db}", echo=True)
    df = pd.read_sql_query("select * from availability where number = %(number)s", engine, params={"number":
station_id})
  
  df['last_update'] = pd.to_datetime(df.last_update, unit='ms')
  df.set_index('last_update', inplace=True)
  res = df['available_bikes'].resample('1H').mean()
  res_stands = df['available_stands'].resample('1H').mean()

  hourly_res = []
  for hour in range(24):
      hourly_res.append([hour,
                         res[res.index.hour == hour].mean(),
                         res_stands[res_stands.index.hour == hour].mean()])
  return jsonify(data=hourly_res)
""""